In [1]:
import pandas as pd
import numpy as np
import networkx as nx
#from sklearn.cluster import SpectralClustering
#from sklearn import metrics
import random
import plotly
import os

# for data aggregation.
import numpy as np
import pandas as pd
import geopandas as gpd
from geopy.distance import distance, lonlat

# for data visualisation.
#import plotly_express as px
import plotly.plotly as py
import cufflinks as cf 
cf.go_offline(connected=True)
cf.set_config_file(theme='polar')
import deckgljupyter.Layer as deckgl

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from tqdm import tqdm_notebook
plotly.__version__


access_token = 'pk.eyJ1IjoiZGFlZG9sIiwiYSI6ImNqZGpqbmpnYzFscm8yd245YXM5MWQxeGgifQ.ACxKlSjUthNpixmVX2faMw'
view_options = {
    'center': [127.021633, 37.248900],
    'zoom': 11,
    'bearing': 0,
    'pitch': 20,
    'style': 'mapbox://styles/mapbox/dark-v9',
    'access_token': access_token
}

In [2]:
import pandas as pd
import numpy as np
import networkx as nx
#from sklearn.cluster import SpectralClustering
#from sklearn import metrics
import random
import plotly
import os

# for data aggregation.
import numpy as np
import pandas as pd
import geopandas as gpd
from geopy.distance import distance, lonlat

# for data visualisation.
#import plotly_express as px
import plotly.plotly as py
import cufflinks as cf 
cf.go_offline(connected=True)
cf.set_config_file(theme='polar')
import deckgljupyter.Layer as deckgl

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from tqdm import tqdm_notebook
plotly.__version__


access_token = 'pk.eyJ1IjoiZGFlZG9sIiwiYSI6ImNqZGpqbmpnYzFscm8yd245YXM5MWQxeGgifQ.ACxKlSjUthNpixmVX2faMw'
view_options = {
    'center': [127.021633, 37.248900],
    'zoom': 11,
    'bearing': 0,
    'pitch': 20,
    'style': 'mapbox://styles/mapbox/dark-v9',
    'access_token': access_token
}
##################################################################################################

df_route_station = pd.read_csv('../data/PJT001_stations_table.csv')
df_route_station.columns = ['STATION_ID','시군명','STATION_NM','정류소영문명','정류소번호','중앙차로여부',
                            '관할관청','위치','MAP_LAT','MAP_LON','모바일정류장ID','이비카드정류장ID']

df_route_info = pd.read_csv('../data/PJT001_routestationinfo.csv')
df_route_info.columns = ['seq','ROUTE_ID','ROUTE_NM','SEQ','STATION_NM','STATION_ID','mobile_id']
df_route_info_raw_org=df_route_info.copy()

df_tripchain=pd.read_csv('../data/PJT001_TripChain.csv')
columns_0 =[]
for col in df_route_station.columns: 
    columns_0.append(col.replace('||', ''))
columns=[]
for col in columns_0: 
    columns.append(col.replace("'",''))
df_route_station.columns=columns
df_route_station_org=df_route_station.copy()

df_route_station_org=pd.merge(df_route_info_raw_org,df_route_station_org, how='left')
df_route_station_raw_org=pd.merge(df_route_info_raw_org,df_route_station_org, how='left')

col = ['ROUTE_ID', 'ROUTE_NM', 'ROUTE_TP', '시작정류장', '종료정류장', 'PERM_VOL', 'COM_NM', 'ADMIN_NM']
route_info_start=[]
route_info_end = []
ROUTE_ID=set(df_route_info['ROUTE_ID'])
for r in ROUTE_ID:
    route_info=df_route_info_raw_org[df_route_info_raw_org.iloc[:,1]==r].sort_values(["SEQ"], ascending=[True])
    route_info_start.append(route_info.head(1).iloc[0])
    route_info_end.append(route_info.tail(1).iloc[0])

route_info_start_end=pd.merge(pd.DataFrame(route_info_start)[['seq','ROUTE_ID','STATION_ID','SEQ','STATION_NM']],pd.DataFrame(route_info_end)[['ROUTE_ID','STATION_ID','SEQ','STATION_NM']],on='ROUTE_ID')
route_info_start_end.columns=['SEQ','ROUTE_ID', 'STATION_ID_start', 'SEQ_start','시작정류장', 'STATION_ID_end', 'SEQ_end', '종료정류장']
df_route_info_org=route_info_start_end.copy()


#df_route_info_raw_org.head()
#df_route_info_org.head()

#df_route_station_org.head()

#동탄면에 있는 모든 정류장
route_station = df_route_station_org.loc[df_route_station_org['관할관청']=='경기도 화성시',:]#109개
df_route_station=route_station
#동탄면에 있는 정류장 각각에 지나는 루트 
STARION_ROUTE = []
STATION_ID=set(route_station['STATION_ID'])
for r in STATION_ID:
    route_info=df_route_info[df_route_info['STATION_ID']==r]
    STARION_ROUTE.append([r,set(route_info['ROUTE_ID'])])
STARION_ROUTE=pd.DataFrame(STARION_ROUTE)
STARION_ROUTE.columns=['STATION_ID', 'ROUTE_ID_set']

#동탄면에 있는 정류장을 하나라도 지나는 모든 루트
ROUTE=pd.merge(route_station[['STATION_ID']],df_route_info,how='left') #528개
ROUTE=pd.DataFrame(ROUTE.dropna()) #514개 (14개는 정류장은 있는데 루트가 없음)
df_route_info_raw=ROUTE.drop_duplicates().copy() #중복 행 발견, 제거 후 469개

# 동탄면 안에서 출발 종료 정류장 정보
col = ['ROUTE_ID', 'ROUTE_NM', 'ROUTE_TP', '시작정류장', '종료정류장', 'PERM_VOL', 'COM_NM', 'ADMIN_NM']
route_info_start=[]
route_info_end = []
ROUTE_ID=set(df_route_info_raw['ROUTE_ID'])
for r in ROUTE_ID:
    route_info=df_route_info_raw[df_route_info_raw["ROUTE_ID"]==r].sort_values(["SEQ"], ascending=[True])
    route_info_start.append(route_info.head(1).iloc[0])
    route_info_end.append(route_info.head(int(len(route_info)/2)).tail(1).iloc[0])
    route_info_end.append(route_info.head(int(len(route_info))).tail(1).iloc[0])


route_info_start_end=pd.merge(pd.DataFrame(route_info_start)[['ROUTE_NM','seq','ROUTE_ID','STATION_ID','SEQ','STATION_NM']],pd.DataFrame(route_info_end)[['ROUTE_ID','STATION_ID','SEQ','STATION_NM']],on='ROUTE_ID')

route_info_start_end.columns=['ROUTE_NM','SEQ','ROUTE_ID', 'STATION_ID_start', 'SEQ_start','시작정류장', 'STATION_ID_end', 'SEQ_end', '종료정류장']
df_route_info=route_info_start_end.copy()

#########################################################################
stations=df_route_station[df_route_station['ROUTE_ID']==241491011][['MAP_LAT','MAP_LON']]
data = [{'position': [lon, lat]} for lat, lon in np.array(stations)]
m = deckgl.Map(**view_options)
m.add(deckgl.GridCellLayer(data,
                           cellSize=100,
                           getElevation=70,
                           getPosition='obj => obj.position',
                           getColor=[255,255,0]
                           ))
m.show()
#############################################################################################


'3.10.0'

In [11]:
set(df_route_station['ROUTE_ID'])

{228000204,
 233000004,
 233000005,
 233000010,
 233000013,
 233000015,
 233000019,
 233000021,
 233000022,
 233000024,
 233000025,
 233000026,
 233000027,
 233000028,
 233000029,
 233000030,
 233000034,
 233000035,
 233000036,
 233000038,
 233000040,
 233000041,
 233000043,
 233000044,
 233000045,
 233000046,
 233000047,
 233000048,
 233000049,
 233000051,
 233000052,
 233000056,
 233000057,
 233000058,
 233000059,
 233000060,
 233000062,
 233000064,
 233000067,
 233000075,
 233000077,
 233000079,
 233000080,
 233000081,
 233000084,
 233000085,
 233000086,
 233000094,
 233000095,
 233000096,
 233000117,
 233000125,
 233000126,
 233000128,
 233000130,
 233000131,
 233000133,
 233000134,
 233000135,
 233000136,
 233000139,
 233000140,
 233000141,
 233000142,
 233000144,
 233000252,
 233000253,
 233000254,
 233000258,
 233000263,
 233000264,
 233000265,
 233000266,
 233000267,
 233000268,
 233000269,
 233000270,
 233000271,
 233000272,
 233000273,
 233000274,
 233000275,
 233000276,
 233

In [13]:
#########################################################################
stations=df_route_station[df_route_station['ROUTE_ID'].isin([241491009,241491010,241491011,233000004,233000004])][['MAP_LAT','MAP_LON']]
data = [{'position': [lon, lat]} for lat, lon in np.array(stations)]
m = deckgl.Map(**view_options)
m.add(deckgl.GridCellLayer(data,
                           cellSize=100,
                           getElevation=70,
                           getPosition='obj => obj.position',
                           getColor=[255,255,0]
                           ))
m.show()
#############################################################################################


In [3]:
def cluster_visualization(df):
    stations = df.iloc[:,2:5]
    data = []
    print(stations.head())
    colors = [] 
    len_color = len(stations['label'].unique())
    for _ in range(len_color):
        colors.append([random.randrange(0,255),random.randrange(0,255),random.randrange(0,255)])
    for i in range(len(stations)):
        label,lat,lon = stations.iloc[i,:]
        data.append({'coordinates': [lon, lat], 'values':100*(label+1),'color':colors[int(label)]})
    return data
data = cluster_visualization(df)


NameError: name 'df_' is not defined